In [1]:
 !pip install tweepy
 !pip install credentials
 !pip install textblob

In [2]:

import tweepy           
import pandas as pd     
import numpy as np   

# For plotting and visualization:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
## specify ur api credentials
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXX'

In [4]:
# Now we define function that can authenticate and extract tweeter data

from credentials import *    # This will allow us to use the keys as variables

# API's setup:
def twitter_setup():
    
    # Utility function to setup the Twitter's API with our access keys provided.
    # Authentication and access using keys:

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    # Return API with authentication:
    
    api = tweepy.API(auth)
    return api

In [5]:
# Create an extractor object which is called tweets:
extractor = twitter_setup()

# We create a tweet list as follows:
tweets = extractor.user_timeline(screen_name="IBM", count=200)
print("Number of tweets extracted: {}.\n".format(len(tweets)))

# We print the most recent 5 tweets:
print("5 recent tweets:\n")
for tweet in tweets[:5]:
    print(tweet.text)
    print()

Number of tweets extracted: 200.

5 recent tweets:

@amour_de_porc Have a great day Penny!

Meet Gerd Binnig and Heinrich Rohrer, inventors of the scanning tunneling microscope. 

Their work opened the door… https://t.co/Rgbsk3ICid

@sfiloteo Thank you Sherrylyn!

@sudo_sudoka Thank you for bringing this to our attention. We will forward to the team that owns the page.

@MoRebaie @MoRebaie https://t.co/CDjmtR9A2p



In [6]:
# We create a pandas dataframe as follows:
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data.head(10))

,Tweets
0,@amour_de_porc Have a great day Penny!
1,"Meet Gerd Binnig and Heinrich Rohrer, inventor..."
2,@sfiloteo Thank you Sherrylyn!
3,@sudo_sudoka Thank you for bringing this to ou...
4,@MoRebaie @MoRebaie https://t.co/CDjmtR9A2p
5,@Honey_Dropz Happy holidays! 💙 ❄️ ☃️
6,@barbphillips77 We wish you and everyone at @W...
7,@pantone 💙
8,@MariamMadiha @MariamMadiha https://t.co/0Miit...
9,@Deepakkoshal 🎉 https://t.co/ID8ftz8s6T


In [7]:
#Creating a data frame
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])


display(data.head(20))

,Tweets
0,@amour_de_porc Have a great day Penny!
1,"Meet Gerd Binnig and Heinrich Rohrer, inventor..."
2,@sfiloteo Thank you Sherrylyn!
3,@sudo_sudoka Thank you for bringing this to ou...
4,@MoRebaie @MoRebaie https://t.co/CDjmtR9A2p
5,@Honey_Dropz Happy holidays! 💙 ❄️ ☃️
6,@barbphillips77 We wish you and everyone at @W...
7,@pantone 💙
8,@MariamMadiha @MariamMadiha https://t.co/0Miit...
9,@Deepakkoshal 🎉 https://t.co/ID8ftz8s6T


In [11]:
# We print info from the first tweet:
print(tweets[0].id)
print(tweets[0].favorite_count)
print(tweets[0].retweet_count)


1204487952119848960
1
0


In [12]:
# We add relevant data:
data['len']  = np.array([len(tweet.text) for tweet in tweets])
data['ID']   = np.array([tweet.id for tweet in tweets])
data['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
data['RTs']    = np.array([tweet.retweet_count for tweet in tweets])

In [13]:
#Simple Statistics to perform
mean = np.mean(data['len'])

print("The average length of tweets: {}".format(mean))

The avarage length of tweets: 68.81


In [23]:
mean=np.mean(data['Likes'])
print("The average length of tweets: {}".format(mean))

The average length of tweets: 22.86


In [14]:
# We extract the tweet with more FAVs and more RTs:

fav_max = np.max(data['Likes'])
rt_max  = np.max(data['RTs'])

fav = data[data.Likes == fav_max].index[0]
rt  = data[data.RTs == rt_max].index[0]

# Max FAVs:
print("The tweet with more likes is: \n{}".format(data['Tweets'][fav]))
print("Number of likes: {}".format(fav_max))
print("{} characters.\n".format(data['len'][fav]))

# Max RTs:
print("The tweet with more retweets is: \n{}".format(data['Tweets'][rt]))
print("Number of retweets: {}".format(rt_max))
print("{} characters.\n".format(data['len'][rt]))

The tweet with more likes is: 
Cloud computing has been around longer than you think: in the 1970s we released the IBM System/370, with one of the… https://t.co/MZ4WGCFFne
Number of likes: 975
140 characters.

The tweet with more retweets is: 
RT @IBM: Blockchain on the IBM Cloud helps everyone from growers to grocers trace food supplies, preventing food waste during recalls. #Sma…
Number of retweets: 211
140 characters.



In [16]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    
    # Utility function to clean the text in a tweet by removing links and special characters using regex.

    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [17]:
def analyze_sentiment(tweet):

   #Utility function to classify the polarity of a tweet using textblob.

    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [25]:
data.head(15)

,Tweets,len,ID,Likes,RTs,SA
0,@amour_de_porc Have a great day Penny!,38,1204487952119848960,1,0,1
1,"Meet Gerd Binnig and Heinrich Rohrer, inventor...",139,1204437304196108288,36,17,0
2,@sfiloteo Thank you Sherrylyn!,30,1204430069785120768,0,0,0
3,@sudo_sudoka Thank you for bringing this to ou...,106,1204426429120876544,1,0,0
4,@MoRebaie @MoRebaie https://t.co/CDjmtR9A2p,43,1204131879005196289,1,0,0
5,@Honey_Dropz Happy holidays! 💙 ❄️ ☃️,36,1204129601905188869,0,0,1
6,@barbphillips77 We wish you and everyone at @W...,93,1204127495903158273,2,0,1
7,@pantone 💙,10,1204126318100713472,3,0,0
8,@MariamMadiha @MariamMadiha https://t.co/0Miit...,51,1204123703690321920,1,0,0
9,@Deepakkoshal 🎉 https://t.co/ID8ftz8s6T,39,1204123271106613250,1,0,0


In [19]:
# We create a column with the result of the analysis:
data['SA'] = np.array([ analyze_sentiment(tweet) for tweet in data['Tweets'] ])


display(data.head(10))

,Tweets,len,ID,Likes,RTs,SA
0,@amour_de_porc Have a great day Penny!,38,1204487952119848960,1,0,1
1,"Meet Gerd Binnig and Heinrich Rohrer, inventor...",139,1204437304196108288,36,17,0
2,@sfiloteo Thank you Sherrylyn!,30,1204430069785120768,0,0,0
3,@sudo_sudoka Thank you for bringing this to ou...,106,1204426429120876544,1,0,0
4,@MoRebaie @MoRebaie https://t.co/CDjmtR9A2p,43,1204131879005196289,1,0,0
5,@Honey_Dropz Happy holidays! 💙 ❄️ ☃️,36,1204129601905188869,0,0,1
6,@barbphillips77 We wish you and everyone at @W...,93,1204127495903158273,2,0,1
7,@pantone 💙,10,1204126318100713472,3,0,0
8,@MariamMadiha @MariamMadiha https://t.co/0Miit...,51,1204123703690321920,1,0,0
9,@Deepakkoshal 🎉 https://t.co/ID8ftz8s6T,39,1204123271106613250,1,0,0


In [20]:
  # We construct lists with classified tweets:

pos_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] < 0]

In [21]:
print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(data['Tweets'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(data['Tweets'])))
print("Percentage of negative tweets: {}%".format(len(neg_tweets)*100/len(data['Tweets'])))

Percentage of positive tweets: 48.0%
Percentage of neutral tweets: 47.5%
Percentage of negative tweets: 4.5%


In [28]:
#Negative words(Twitter & Term)

print(neg_tweets)

["@Tolwyn Hi there, we're sorry for your frustration. We would recommend bookmarking the main IBM Support page and na… https://t.co/JIxETN725N", 'Maintaining a strong wireless network is difficult since physical structures can block wavelengths. \n\nCheck out how… https://t.co/peNv3JI7tW', '@wnba_youngboy Ouch. Hangovers are no fun at all 😟. In future, limit your alcohol intake and stay hydrated 💧💧💧', "@densityMeter Hi Brian, we're sorry for your frustration. Please visit our web page and open a support case here so… https://t.co/cf7GShUIg5", '@martinlieberman Thank you for your hard work Martin!', '@Duv09 Thankful for your grandfathers hard work and innovation! Thanks for sharing with us David.', 'RT @IBM: Blockchain on the IBM Cloud helps everyone from growers to grocers trace food supplies, preventing food waste during recalls. #Sma…', '@JohanSjareltjen Thank you for your hard work! #ProudIBMer', '@cascholten Thanks for your hard work Clark!']


In [46]:

import pandas as pd
import numpy as np


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV


from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

import numpy as np
import matplotlib.pyplot as plt


In [50]:
df=data['SA']


In [51]:
print(df)

0      1
1      0
2      0
3      0
4      0
      ..
195    0
196    1
197    0
198    0
199    1
Name: SA, Length: 200, dtype: int32


In [52]:

#Medavakkam wikipedia bag of words

import nltk  
import numpy as np  
import random  
import string

import bs4 as bs  
import urllib.request  
import re  

In [81]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Medavakkam')  
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:  
    article_text += para.text

In [82]:
corpus = nltk.sent_tokenize(article_text)

In [83]:
for i in range(len(corpus )):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i])
    corpus [i] = re.sub(r'\s+',' ',corpus [i])


In [97]:
print(corpus[2])


this location is a fast developing residential locality due to its close proximity to the it corridor omr approx 4 km from sholinganallur and sez in medavakkam sholinganallur road 


In [98]:
wordfreq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

In [99]:
print(wordfreq)

{'medavakkam': 7, 'is': 5, 'a': 3, 'rural': 1, 'village': 2, 'in': 6, 'tambaram': 3, 'taluk': 1, 'of': 5, 'chengalpattu': 1, 'district': 1, 'which': 2, 'adjacent': 1, 'to': 6, 'the': 11, 'neighbourhood': 2, 'sholinganallur': 4, 'madipakkam': 1, 'selaiyur': 1, 'keelkattalai': 1, 'velachery': 2, 'and': 6, 'it': 2, 'center': 1, 'place': 1, 'developing': 2, 'suburbs': 1, 'like': 1, 'ponmar': 1, 'ottiyambakkam': 1, 'sithalapakkam': 1, 'administered': 1, 'by': 3, 'panchayat': 1, 'comes': 1, 'under': 1, 'shollinganallur': 1, 'mla': 1, 'constituency': 2, 'south': 1, 'chennai': 1, 'mp': 1, 'this': 2, 'location': 1, 'fast': 1, 'residential': 3, 'locality': 1, 'due': 1, 'its': 1, 'close': 1, 'proximity': 1, 'corridor': 1, 'omr': 1, 'approx': 1, '4': 1, 'km': 1, 'from': 3, 'sez': 1, 'road': 5, 'some': 1, 'areas': 2, 'are': 3, 'vadakupattu': 1, 'annai': 1, 'indira': 1, 'nagar': 18, 'babu': 1, 'somu': 1, 'munusamy': 1, 'vimala': 1, 'vgp': 1, 'pushpa': 1, 'aishwarya': 2, 'gardens': 1, 'garden': 3, 'e

In [100]:
import heapq
most_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)

In [101]:
bag_words = []
for sentence in corpus:
    sentence_tokens = nltk.word_tokenize(sentence)
    sent_vec = []
    for token in most_freq:
        if token in sentence_tokens:
            sent_vec.append(1)
        else:
            sent_vec.append(0)
    bag_words.append(sent_vec)

In [91]:
print(bag_words)

[[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0